In [ ]:

import os, re
import boto3
import matplotlib.pyplot as plt
import numpy as np
import sagemaker
from sagemaker.amazon.common import numpy_to_record_serializer
from sagemaker.predictor import csv_serializer, json_deserializer
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [ ]:
bucket = 'sagemaker-us-east-2-343904395757' # modify to your bucket name
prefix = 'knn-better-reads'
key = 'recordio-pb-data'

In [ ]:
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)

In [ ]:
output_path = 's3://' + bucket + '/' + prefix + '/default_example/output'

In [ ]:
hyperparams = {
    'feature_dim': 1038,
    'k': 10,
    'sample_size': 24989,
    'predictor_type': 'classifier' 
}

In [ ]:
knn = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "knn"),
        get_execution_role(),
        train_instance_count=1,
        train_instance_type='ml.m4.4xlarge',
        output_path=output_path,
        sagemaker_session=sagemaker.Session())
knn.set_hyperparameters(**hyperparams)

In [ ]:
finput = {'train': s3_train_data}
knn.fit(finput)

In [ ]:
knn_predict = knn.deploy(initial_instance_count=1,
                         instance_type='ml.t2.medium')